In [21]:
import os
import numpy as np
import pandas as pd
import codecs

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from datetime import datetime

■■■■■　関数　■■■■■

In [3]:
def SaveFig(fig_id, tight_layout=True, fig_extension="png", resolution=600):
   
    images_path = os.path.join("..", "Images", csv_file[:-4])
    os.makedirs(images_path, exist_ok=True)
    file_name = os.path.join(images_path, fig_id + "." + fig_extension)
    
    #print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(file_name, format=fig_extension, dpi=resolution)
    plt.close()

In [4]:
def GetCSVFiles(path):
    files = os.listdir(path)
    csv_files = []
    
    for file in files:
        if(file.find(".csv") != -1 and file[:2].upper() == "NG"):
             csv_files.append(file)
    
    return csv_files

In [5]:
def GetExtractCSVFiles(path):
    files = os.listdir(path)
    csv_files = []
    
    for file in files:
        if(file.find(".csv") != -1 and file.upper().find("EXTRACT") != -1):
             csv_files.append(file)
    
    return csv_files

In [6]:
def CreateBaseData(csv_file):
    
    #工程番号定義
    process_no = [11, 12, 13, 14, 15, 16]
    
    #取得列名の定義
    col_output  = ["out_put_time_%s"%i for i in process_no]
    col_others  = ["sa_lot", "sa_seri", "ピックアップ位置XY_11", "格納位置_14", "格納時間_14", "排出時間_14", "不良カテゴリ_16"]
    col_rename  = {"ピックアップ位置XY_11": "pickup_loc_XY_11",
                   "格納位置_14": "storage_loc_14",
                   "格納時間_14": "storage_start_14",
                   "排出時間_14": "storage_end_14",
                   "不良カテゴリ_16": "defective_cat_16"}
    col_process = col_others + col_output
    
    #指定工程Noのデータ読み込み
    with codecs.open(os.path.join(path, csv_file), "r", "Shift-JIS", "ignore") as f:
        df = pd.read_table(f, delimiter=",", low_memory=False, usecols=col_process)

    #列名をリネーム
    df = df.rename(columns=col_rename)
    
    #日付：オブジェクト　→　datetimeへ変換
    for i in process_no:
        df["out_put_time_%s"%i] = pd.to_datetime(df["out_put_time_%s"%i])
    
    df.to_csv(os.path.join(path, "Extract_" + csv_file), index=False, encoding="utf-8")

In [24]:
def PlotMatrixData(csv_file):
    
    #ExtractCSV読み込み
    df = pd.read_csv(os.path.join(path, csv_file))
    lots = df["sa_lot"].unique()
    lots = np.sort(lots)
    
    #1画像辺りのサブプロット数
    plt_row = 3
    plt_col = 6
    
    #figNo設定
    i_mat = 0
    i_fig = 0
        
    #部屋番号作成
    room_row = np.arange(100, 1600, 100)
    room_col = np.r_[np.arange(1,5), np.arange(6,10)]
    
    #plt.figure(figsize = (10, 8))
    
    for lot in lots:
        
        i_mat += 1
        
        if i_mat == 1:
            plt.figure(figsize=(10, 8))
        
        #カウントアップ変数
        defect_num = np.zeros([len(room_row), len(room_col)], dtype = int)
    
        #31番の不良選定
        df_defect = df[df["defective_cat_16"]==31]
        df_lot    = df_defect[df_defect["sa_lot"]==lot]
    
        #部屋毎の不良数カウント
        for row in room_row:
            for col in room_col:
                room_no = row + col
            
                df_room = df_lot["storage_loc_14"] == room_no
                cnt = df_room.sum()
            
                if cnt != 0:
                    r = room_no // 100 - 1
                    c = room_no % 100 - np.where((room_no % 100) < 5, 1, 2)
                
                    defect_num[r][c] = cnt
    
        #プロット
        plt.subplot(plt_row, plt_col, i_mat)
       # PlotBarRoomDefectives(room_row, room_col, defect_num, lot)
        PlotMatrixRoomDefectives(room_row, room_col, defect_num, lot)
               
        if (i_mat == plt_row * plt_col) or (lot == lots[-1]):
            i_mat = 0
            i_fig += 1
            plt.tight_layout()
            SaveFig(str(i_fig))
            plt.close()

In [8]:
def PlotBarRoomDefectives(room_row, room_col, defect_num, lot):
    
    room_no = (room_row.reshape(-1,1) + room_col).flatten()
    
    df_count = pd.DataFrame({"storage_no" : room_no, "count":defect_num.flatten()}, columns=["storage_no", "count"])
    df_count.plot(x="storage_no", y="count", kind="bar", color="gray", figsize=(16,6))

In [18]:
def PlotMatrixRoomDefectives(room_row, room_col, defect_num, lot):
    
    #メッシュでプロットこんな感じ
    X, Y = np.meshgrid(room_row, room_col)
    Z = defect_num
    
    #plt.figure()
    plt.imshow(Z,interpolation="None", cmap="Reds")
    
    xtick = room_col
    ytick = room_row
    
    locx = np.arange(len(room_col))
    locy = np.arange(len(room_row))
    
    plt.xticks(locx, xtick)
    plt.yticks(locy, ytick)
    
    plt.tick_params(labelsize=8)
    
    plt.title(lot)

■■■■■　MainProgram　■■■■■

In [10]:
#定義
path = os.path.join("..", "AssemblyData")

In [12]:
#データファイル取得
csv_files = GetExtractCSVFiles(path)

In [23]:
#メイン
for csv_file in csv_files:
    PlotMatrixData(csv_file)

■■■■■　確認　■■■■■

In [15]:
csv_file = csv_files[0]

In [29]:
CreateBaseData(csv_file)

■■■■■　失敗作　■■■■■